# MMF1921 (Summer 2025) - Project 1
 
 The purpose of this program is to implement the following factor models
 
     a) Multi-factor OLS regression
     b) Fama-French 3-factor model
     c) LASSO
     d) Best Subset Selection
 
 and to use these factor models to estimate the asset expected returns and covariance matrix. 
 
These parameters will then be used to test the out-of-sample performance using MVO to construct optimal portfolios.
 
 Use can use this template to write your program.

     Student Name:
     Student ID:

In [1]:
import time
import math
from scipy.stats import gmean
import matplotlib.pyplot as plt
from functions.BSS import *
from functions.FF import *
from functions.LASSO import *
from functions.MVO import *
from functions.OLS import *
import pandas as pd

adjClose = pd.read_csv("MMF1921_AssetPrices.csv", index_col=0)
factorRet = pd.read_csv("MMF1921_FactorReturns.csv", index_col=0)

In [2]:
adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

In [3]:
#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:,factorRet.columns != 'RF']

In [4]:
#Identify the tickers and the dates
tickers = adjClose.columns
dates   = factorRet.index

In [5]:
# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]

returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:,:]

In [6]:
assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]

In [8]:
factorRet

,Mkt_RF,SMB,HML,RMW,CMA,Mom,ST_Rev,LT_Rev
Date,,,,,,,,
2006-01-31,0.0304,0.0577,0.0112,-0.0094,-0.0054,0.0268,-0.0110,0.0115
2006-02-28,-0.0030,-0.0045,-0.0025,-0.0070,0.0200,-0.0184,0.0335,0.0125
2006-03-31,0.0146,0.0345,0.0060,-0.0004,-0.0050,0.0126,0.0003,0.0048
2006-04-30,0.0073,-0.0083,0.0260,0.0115,-0.0025,0.0061,-0.0036,-0.0047
2006-05-31,-0.0357,-0.0299,0.0255,0.0096,0.0134,-0.0365,-0.0003,-0.0017
...,...,...,...,...,...,...,...,...
2016-08-31,0.0050,0.0172,0.0334,-0.0133,-0.0039,-0.0312,-0.0034,0.0054
2016-09-30,0.0025,0.0174,-0.0149,-0.0229,-0.0008,-0.0058,0.0009,0.0178
2016-10-31,-0.0202,-0.0401,0.0416,0.0119,0.0024,0.0065,0.0272,-0.0111


In [7]:
returns

,F,CAT,DIS,MCD,KO,PEP,WMT,C,WFC,JPM,AAPL,IBM,PFE,JNJ,XOM,MRO,ED,T,VZ,NEM
Date,,,,,,,,,,,,,,,,,,,,
2006-01-31,0.120718,0.176565,0.052403,0.034756,0.023044,-0.035660,-0.018244,-0.043681,-0.010981,0.006606,0.046855,-0.014449,0.097701,-0.046096,0.113644,0.257284,0.011177,0.070249,0.061208,0.153803
2006-02-28,-0.074496,0.072889,0.102487,-0.006256,0.010858,0.030353,-0.019665,0.002888,0.034763,0.031569,-0.096368,-0.013995,0.025742,0.004236,-0.052251,-0.080411,-0.015327,0.059798,0.061035,-0.147089
2006-03-31,-0.004955,-0.021078,-0.007273,-0.019455,0.001196,-0.021782,0.041595,0.014846,-0.008840,0.008454,-0.087946,0.024092,-0.052192,0.023533,0.021397,0.075195,-0.055368,-0.023635,0.006982,-0.021234
2006-04-30,-0.117922,0.054510,-0.001090,0.002512,-0.001450,0.004187,-0.050382,0.064753,0.071866,0.094983,0.118690,-0.005176,0.012853,-0.013901,0.032877,0.038280,-0.012336,-0.022110,-0.022179,0.121087
2006-05-31,0.025916,-0.041137,0.086544,-0.044798,0.045033,0.033818,0.075497,-0.017313,-0.030615,-0.064679,-0.155174,-0.030421,-0.061277,0.029544,-0.033864,-0.053583,0.032254,-0.010023,-0.059401,-0.110708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-31,-0.004939,-0.009987,-0.015729,-0.009530,-0.004784,-0.013214,-0.014548,0.089506,0.067255,0.054982,0.023418,-0.002487,-0.048836,-0.040876,-0.012046,0.104668,-0.052257,-0.055871,-0.055786,-0.131109
2016-09-30,-0.042263,0.083021,-0.017138,-0.002794,-0.017692,0.018723,0.009318,-0.010883,-0.128546,-0.013681,0.065304,-0.000389,-0.026924,-0.010339,0.001407,0.052397,0.000464,-0.006805,-0.006888,0.027916
2016-10-31,-0.015228,-0.051661,-0.002031,-0.024385,0.001690,-0.014634,-0.029318,0.040452,0.038870,0.047392,0.004134,-0.032683,-0.063973,-0.018316,-0.045571,-0.166550,0.003120,-0.083161,-0.064214,-0.057466


# 2. Define your initial parameters

In [ ]:
#Initial budget to invest ($100,000)
initialVal = 100000

#Start of in-sample calibration period
calStart = pd.to_datetime('2008-01-01', format = '%Y-%m-%d')
calEnd   = calStart + pd.offsets.DateOffset(years=4) - pd.offsets.DateOffset(days=1)

#Start of out-of-sample test period
testStart = pd.to_datetime('2012-01-01', format = '%Y-%m-%d')
testEnd   = testStart + pd.offsets.DateOffset(years=1) - pd.offsets.DateOffset(days=1)

#Number of investment periods (each investment period is 1 year long)
NoPeriods = 5

#Factor models
#Note: You must populate the functions OLS.py, FF.py, LASSO.py and BSS.py with your own code.
FMList = [OLS, FF, LASSO, BSS]

NoModels = len(FMList)

#Tags for the portfolios under the different factor models
tags = ['OLS portfolio', 'FF portfolio', 'LASSO portfolio', 'BSS portfolio']

# 3. Construct and rebalance your portfolios

Here you will estimate your input parameters (exp. returns and cov. matrix etc) from the Fama-French factor models.
You will have to re-estimate your parameters at the start of each rebalance period, and then re-optimize and rebalance your portfolios accordingly.

Ensure you re-initialize the dates above if you run this cell repeatedly. 

In [ ]:
# Initiate counter for the number of observations per investment period
toDay = 0

# Preallocate the space for the per period value of the portfolios 
currentVal = {i:np.zeros(NoPeriods) for i in range(NoModels)}

# Number of assets
n  = len(tickers)

# Preallocate space for the portfolio weights
x  = {i:np.zeros([n, NoPeriods]) for i in range(NoModels)}

# Initialize dictionaries to hold Q, mu and the number of shares 
# for each model. These are overwritten at each rebalancing point
mu = {}
Q = {}
NoShares ={}

# Empty lists to measure the value of the portfolio over the period
portfValue = {i:[] for i in range(NoModels)}

#--------------------------------------------------------------------------
# Set the value of lambda and K for the LASSO and BSS models, respectively
#--------------------------------------------------------------------------
# lambda_ = 
# K    = 

for t in range(NoPeriods):
    # Subset the returns and factor returns corresponding to the current calibration period.
    periodReturns = returns[(calStart <= returns.index)&(returns.index <= calEnd)]
    periodFactRet = factorRet[(calStart <= factorRet.index) &(factorRet.index <= calEnd)]

    current_price_idx = (calEnd - pd.offsets.DateOffset(days = 7) <= adjClose.index)&(adjClose.index <= calEnd)
    currentPrices = adjClose[current_price_idx]
    
    # Subset the prices corresponding to the current out-of-sample test period.
    periodPrices_idx = (testStart <= adjClose.index)&(adjClose.index <= testEnd)
    periodPrices = adjClose[periodPrices_idx]

    assert len(currentPrices) == 1
    # Set the initial value of the portfolio or update the portfolio value
    if t == 0:
        for i in range(NoModels):
            currentVal[i][0] = initialVal # all models start with the same amount of $
    else:
        for i in range(NoModels):
            currentVal[i][t] = (currentPrices @  NoShares[i].values.T).squeeze()
    
    # Update counter for the number of observations per investment period
    fromDay = toDay
    toDay   = toDay + len(periodPrices)

    # Calculate 'mu' and 'Q' using the 4 factor models.
    # Note: You need to write the code for the 4 factor model functions. 
    for i in range(NoModels):
        
        mu[i], Q[i] = FMList[i](periodReturns, periodFactRet, lambda_, K)
            
    # Optimize your portfolios to get the weights 'x'
    # Note: You need to write the code for MVO with no short sales
    for i in range(NoModels):
        
        # Define the target return as the geometric mean of the market 
        # factor for the current calibration period
        targetRet = gmean(periodFactRet.iloc[:,0] + 1) - 1 
        
        x[i][:,t] = MVO(mu[i], Q[i], targetRet) 

    # Calculate the optimal number of shares of each stock you should hold
    for i in range(NoModels):
        
        # Number of shares your portfolio holds per stock
        NoShares[i] = x[i][:,t]*currentVal[i][t]/currentPrices
        
        # Weekly portfolio value during the out-of-sample window
        portfValue[i].append(periodPrices @ NoShares[i].values.T)
        

    # Update your calibration and out-of-sample test periods
    calStart = calStart + pd.offsets.DateOffset(years=1)
    calEnd   = calStart + pd.offsets.DateOffset(years=4) - pd.offsets.DateOffset(days=1)
    
    testStart = testStart + pd.offsets.DateOffset(years=1)
    testEnd   = testStart + pd.offsets.DateOffset(years=1) - pd.offsets.DateOffset(days=1)
    
for i in range(NoModels):
    portfValue[i] = pd.concat(portfValue[i], axis = 0)
    
# Overwrite into a dataframe
portfValue = pd.DataFrame([portfValue[i].values.squeeze() for i in range(NoModels)], 
                          index = tags, columns = portfValue[0].index).T


# 4. Results

In [ ]:
#--------------------------------------------------------------------------
# 4.1 Evaluate any measures of fit of the regression models to assess their
# in-sample quality. You may want to modify Section 3 of this program to
# calculate the quality of fit each time the models are recalibrated.
#--------------------------------------------------------------------------

#--------------------------------------------------------------------------
# 4.2 Calculate the portfolio average return, variance (or standard 
# deviation), and any other performance and/or risk metric you wish to 
# include in your report.
#--------------------------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------------
# 4.3 Plot the portfolio wealth evolution 
# 
# Note: The code below plots all portfolios onto a single plot. However,
# you may want to split this into multiple plots for clarity, or to
# compare a subset of the portfolios. 
#--------------------------------------------------------------------------
# Calculate the dates of the out-of-sample period

fig = plt.figure(1)
portfValue.plot(title = 'Portfolio wealth evolution',
                ylabel = 'Total wealth',
                figsize = (6, 3),
                legend = True)
plt.savefig("images/wealth.svg")

#--------------------------------------------------------------------------
# 4.4 Plot the portfolio weights period-over-period
#--------------------------------------------------------------------------
# OLS Portfolio weights

fig2 = plt.figure(2)
x[0][x[0] < 0] = 0
weights = pd.DataFrame(x[0][(x[0] >0).any(axis=1)], index = tickers[(x[0] >0).any(axis=1)])
weights.columns = [col + 1 for col in weights.columns]
weights.T.plot.area(title = 'Portfolio weights',
                  ylabel = 'Weights', xlabel = 'Rebalance Period',
                  figsize = (6, 3),
                  legend = True, stacked = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.savefig("images/weights.svg")
#
# ###########################################################################
# # Program End